<a href="https://colab.research.google.com/github/pinilDissanayaka/Skin-Diseases-Image-Classification/blob/main/Skin_Diseases_Using_Resnet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [2]:
! kaggle datasets download haroonalam16/20-skin-diseases-dataset

 97% 310M/321M [00:02<00:00, 171MB/s]
100% 321M/321M [00:02<00:00, 142MB/s]


In [4]:
! unzip /content/20-skin-diseases-dataset.zip

Archive:  /content/20-skin-diseases-dataset.zip
replace Dataset/test/Acne and Rosacea Photos/07PerioralDermEye.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings(action = 'ignore')

In [6]:
IMG_SIZE = [224, 224, 3]
basedir = '/content/Dataset'

In [7]:
traindir = basedir + '/' + 'train'
testdir = basedir + '/' + 'test'

In [8]:
print(f"Traing set has {len(os.listdir(traindir))} folders")
print(f"Testing set has {len(os.listdir(testdir))} folders")

outputsize = len(os.listdir(traindir))

Traing set has 19 folders
Testing set has 20 folders


In [31]:
import shutil

shutil.rmtree('/content/Dataset/test/Bullous Disease Photos')

In [36]:
resnet = ResNet101(include_top= False, weights = 'imagenet', input_shape = IMG_SIZE)

171446536/171446536 [==============================] - 1s 0us/step


In [37]:
for layer in resnet.layers:
  layer.trainable = False

In [38]:
x = Flatten()(resnet.output)

x = Dense(units = 128, activation = 'relu')(x)
x = Dense(units = outputsize, activation = 'softmax')(x)

In [39]:
model = Model(inputs = [resnet.input], outputs = [x])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [40]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

In [41]:
earlystopping = EarlyStopping(monitor='val_loss', verbose=1, patience=1)

In [42]:
traindatagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.3,
    height_shift_range=0.9,
    horizontal_flip=True)

testdatagen = ImageDataGenerator(
    rescale=1./255)

In [43]:
traindf = traindatagen.flow_from_directory(
        traindir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

testdf = traindatagen.flow_from_directory(
        testdir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 2609 images belonging to 19 classes.
Found 895 images belonging to 19 classes.


In [44]:
model_history = model.fit_generator(
    traindf,
    epochs = 5,
    verbose = 1,
    validation_data = testdf,
    steps_per_epoch = len(traindf),
    validation_steps = len(testdf),
    callbacks = [earlystopping]
)

Epoch 1/5
82/82 [==============================] - 76s 848ms/step - loss: 3.5507 - acc: 0.2760 - val_loss: 2.5503 - val_acc: 0.2581
Epoch 2/5
82/82 [==============================] - 62s 760ms/step - loss: 2.4045 - acc: 0.3116 - val_loss: 2.4654 - val_acc: 0.2279
Epoch 3/5
82/82 [==============================] - 65s 790ms/step - loss: 2.3612 - acc: 0.3220 - val_loss: 2.5650 - val_acc: 0.2581
Epoch 3: early stopping
